In [1]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 3.2 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
##This is the multiclass classification problem, create the CNN model

fashion_mnist=keras.datasets.fashion_mnist

In [4]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [5]:
##In case of image it has the pixles lies between 1 to 255 so, i have to scale down it to 0 to 1 so,divide it by 255.0
train_images=train_images/255.0
test_images=test_images/255.0

In [6]:
train_images[0].shape

(28, 28)

In [7]:
train_images=train_images.reshape(len(train_images),28,28,1)       ## image is of pixels 28*28 and no. of channel is 1 i.e it is the Greyscale image
test_images=test_images.reshape(len(test_images),28,28,1)

In [8]:
def build_model(hp):                                 ## hp means hyperparameters
  model = keras.Sequential([                         ## Create the sequential model
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),    ##In the first conv. layers number of filters lies between 32 to 128 min.value is 32 and max. value is 128 and we have to try i.e itterate on all of these to take the best one and take the step size of 16
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),           ## Take either 3*3 or 5*5 as the kernel size
        activation='relu',                                               ##Apply the RELU activation function
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),             ##In the second conv. layers number of filters lies between 32 to 64 min.value is 32 and max. value is 64 and we have to try i.e itterate on all of these to take the best one and take the step size of 16
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),                             ## Take either 3*3 or 5*5 as the kernel size
        activation='relu'
    ),
    keras.layers.Flatten(),                                                          ##Flatten it i.e stacked one over the other
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),        ##In the first hidden layer number of neurons lies between 32 to 128 and we have to check it with the step size of 16 and take the best number of neurons
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')                                  ##In the output layer number of neurons is 10 and take the activation function as the softmax
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3,1e-4])),     ##Choose the learning rate value out of them and apply the Adam optimizer
              loss='sparse_categorical_crossentropy',                              #Since it is the multiclassclassification problem so,apply the metrics as the accuracy and loss as the sparse categorical crossentropy
              metrics=['accuracy'])

  return model

In [9]:
from kerastuner import RandomSearch                                    ##Apply the Hyperparameter tunnning using Randomsearch
from kerastuner.engine.hyperparameters import HyperParameters



<ipython-input-9-2618f17054e6>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import RandomSearch                                    ##Apply the Hyperparameter tunnning using Randomsearch


In [12]:
##Feeding the models and parameters to Random Search

tuner=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")  ##This will run for (5*3=15) times for 3 epochs

In [13]:
##This  tells us how many parameters we are tunning

tuner.search_space_summary()

Search space summary
Default search space size: 6
conv_1_filter (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 16, 'sampling': 'linear'}
conv_1_kernel (Choice)
{'default': 3, 'conditions': [], 'values': [3, 5], 'ordered': True}
conv_2_filter (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 64, 'step': 16, 'sampling': 'linear'}
conv_2_kernel (Choice)
{'default': 3, 'conditions': [], 'values': [3, 5], 'ordered': True}
dense_1_units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 16, 'sampling': 'linear'}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [14]:
##Fitting the tuner on train datasets

tuner.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 23m 12s]
val_accuracy: 0.9133333563804626

Best val_accuracy So Far: 0.9133333563804626
Total elapsed time: 01h 51m 17s


In [15]:
tuner.results_summary()

Results summary
Results in output/Mnist Fashion
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 4 summary
Hyperparameters:
conv_1_filter: 80
conv_1_kernel: 3
conv_2_filter: 32
conv_2_kernel: 5
dense_1_units: 80
learning_rate: 0.001
Score: 0.9133333563804626

Trial 0 summary
Hyperparameters:
conv_1_filter: 112
conv_1_kernel: 3
conv_2_filter: 32
conv_2_kernel: 3
dense_1_units: 80
learning_rate: 0.001
Score: 0.9121666550636292

Trial 1 summary
Hyperparameters:
conv_1_filter: 80
conv_1_kernel: 3
conv_2_filter: 32
conv_2_kernel: 5
dense_1_units: 64
learning_rate: 0.001
Score: 0.9108333587646484

Trial 3 summary
Hyperparameters:
conv_1_filter: 96
conv_1_kernel: 3
conv_2_filter: 64
conv_2_kernel: 5
dense_1_units: 32
learning_rate: 0.001
Score: 0.9081666469573975

Trial 2 summary
Hyperparameters:
conv_1_filter: 48
conv_1_kernel: 5
conv_2_filter: 64
conv_2_kernel: 3
dense_1_units: 128
learning_rate: 0.0001
Score: 0.8928333520889282
